In [ ]:
from urllib.request import urlretrieve
urlretrieve('https://drive.google.com/uc?export=download&id=1xCLdrQzjKzgPhbFZ7x9fe7acmUDX7m-p', 'stop_words.txt')

('stop_words.txt', <http.client.HTTPMessage at 0x7facb62eac90>)

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 49.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=63167d1915a21c1e9a2beff9e95a9ed48159e6138ea074ebe45725bffc5f2907
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("review_sentiment")
sc = SparkContext.getOrCreate( conf =conf)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# read stop words
stop_words_file = open('stop_words.txt')
stop_words = stop_words_file.read().split()
stop_words_file.close()

In [ ]:
#import pyspark.sql.functions
#from pyspark.sql.types import *
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/Grad School/MSBA/Big Data Tech/Reviews.csv")
data = data.tail(500)

In [ ]:
df2 = data[['Id', 'ProductId', 'UserId', 'Text']]

'''
for x in range(len(df2)):
  if df2.iloc[x,3][-1] == ".":
    df2.iloc[x,3] = df2.iloc[x,3][:-2]
'''

df2['Text'] = df2['Text'].str.replace("!", ".")
df2['Text'] = df2['Text'].str.replace("?", ".")

df2 = (df2.set_index(['Id', 'ProductId', 'UserId'])
   .apply(lambda x: x.str.split('.').explode())
   .reset_index())  

df2 = df2.loc[df2['Text'].str.strip().str.len() != 0]

df2['Text'] = df2['Text'].str.strip("<br />")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [ ]:
df2.to_csv(r'/content/drive/MyDrive/Grad School/MSBA/Big Data Tech/processed_reviews2.csv', index = False)